# Sistemi dinamici: Jordanizzazione, evoluzione libera e forzata

## Jordanizzazione (normale)

Il primo esercizio consiste nel mettere un sistema $\Sigma = \{ A, B, C, 0 \}$ nella forma di Jordan $\Sigma_J = \{ A_J, B_J, C_J, 0 \}$. 

Il procedimento consiste nel trovare una matrice di cambiamento di base $T$ tale che la matrice $A$ sia messa in forma di Jordan. 
 - $A_J = T^{-1} A T$
 - $B_J = T^{-1} B$
 - $C_J = C T$

Scrivi di seguito il sistema.

In [1]:
A = sym([4 0 0 -3 0 1; 5 4 3 -2 -2 -2; -5 0 2 3 -3 3; 6 -1 0 -1 1 1; 0 2 3 1 0 0; -1 3 0 -3 -3 3])

 
A =
 
[  4,  0, 0, -3,  0,  1]
[  5,  4, 3, -2, -2, -2]
[ -5,  0, 2,  3, -3,  3]
[  6, -1, 0, -1,  1,  1]
[  0,  2, 3,  1,  0,  0]
[ -1,  3, 0, -3, -3,  3]
 


Iniziamo con un'analisi degli autovalori della matrice. Se gli autovalori hanno tutti molteplicità algebrica uguale a quella geometrica, ce la caviamo easy perchè la matrice è diagonalizzabile. 

Altrimenti, occorre effettivamente mettere in forma di Jordan. Ogni autovalore $\lambda_i$ genera un blocco di dimensione pari alla sua molteplicità algebrica $m_a(\lambda_i)$, con al suo interno tanti mini-blocchi quanto è la sua molteplicità geometrica $m_g(\lambda_i)$.

In [2]:
syms lambda;

fprintf('Il polinomio caratteristico della matrice A è: ');
disp(charpoly(sym(A), lambda));

fprintf('che corrisponde ad avere gli autovalori: ')
eigenvalues = eig(A);
disp(eigenvalues');

Il polinomio caratteristico della matrice A è: lambda^6 - 12*lambda^5 + 87*lambda^4 - 376*lambda^3 + 1131*lambda^2 - 2028*lambda + 2197
 
che corrisponde ad avere gli autovalori: [ 2 + 3i, 2 + 3i, 2 + 3i, 2 - 3i, 2 - 3i, 2 - 3i]
 


Per ogni autovalore, vien comodo costruirsi una tabella con $m_g(\lambda_i)$ righe ed $m_a(\lambda_i)$ colonne. L'intestazione della tabella della colonna k-esima è la differenza tra le dimensioni dei sottospazi $\text{ker}(A - \lambda_i I)^k$ ed $\text{ker}(A - \lambda_i I)^{k-1}$.

La differenza serve a mettere delle 'x' nelle celle corrispondenti alla colonna, partendo dall'alto e scendendo. Il numero delle 'x' da mettere è pari alla differenza della data colonna.

Se contiamo quante 'x' ha ogni riga, troviamo la dimensione dei mini-blocchi di Jordan per il dato blocco.

In [8]:
uniqueEigenvalues = unique(eigenvalues);

jordanBlockSize = [];

for i = 1:numel(uniqueEigenvalues)
    
    lambda = uniqueEigenvalues(i);
    miniblockSize = GetMiniblockSizeOfEigenvalue(lambda, A); % GetMiniblockSizeOfEigenvalue.m
    
    for j = 1:numel(miniblockSize)
        sz = miniblockSize(j);
        jordanBlockSize = [jordanBlockSize ; [lambda sz] ];
    end
end

fprintf('*** Coppie dei mini-blocchi (autovalore, dimensione) ***\n');
disp(jordanBlockSize);


*** Autovalore lambda_i = 2 (ma=3, mg=1) ***
Dimensioni dei miniblocchi:
     3


*** Autovalore lambda_i = 2 (ma=3, mg=1) ***
Dimensioni dei miniblocchi:
     3

*** Coppie dei mini-blocchi (autovalore, dimensione) ***
[ 2 - 3i, 3]
[ 2 + 3i, 3]
 


Ogni mini-blocco di dimensione $k = m_g(\lambda_i)$ aggiunge alla matrice di cambiamento di base $T$ una catena di Jordan, che consiste in tante colonne quante la dimensione $d$. La colonna più a destra sarà un autovettore generalizzato di ordine $k$, le altre colonne sono ottenute invece, da destra verso sinistra, moltiplicando la colonna a destra per $(A - \lambda_i I)$.

In [9]:
[jbr jbc] = size(jordanBlockSize);
[ar ac] = size(A);
n = ac;

AJ = [];
T = [];

for i = 1:jbr
    
    % estraggo le informazioni che mi servono dalla tabella appena costruita
    lambda = jordanBlockSize(i,1);
    order = jordanBlockSize(i,2);

    % costruisco AJ e T
    AJ = blkdiag(AJ, CreateJordanBlock(lambda, order));
    T = [ T, CreateJordanChain(A - lambda*eye(n), order, T) ];
end

fprintf('T = \n');
disp(T);

fprintf('T^{-1} = \n');
disp(inv(T));

fprintf('AJ = \n');
disp(sym(AJ));

fprintf('Controprova: T^{-1} * A * T = \n');
disp(inv(T) * A * T);

T = 
[            0, - 1/2 + 1i/2, 1/2 - 1i/2,            0, - 1/2 - 1i/2, 1/2 + 1i/2]
[ - 1/2 - 1i/2, - 1/2 + 1i/2,          1, - 1/2 + 1i/2, - 1/2 - 1i/2,          1]
[ - 1/2 + 1i/2,   1/2 - 1i/2,         -1, - 1/2 - 1i/2,   1/2 + 1i/2,         -1]
[            0,           -1,          1,            0,           -1,          1]
[ - 1/2 - 1i/2,            0,          0, - 1/2 + 1i/2,            0,          0]
[            0, - 1/2 + 1i/2,          0,            0, - 1/2 - 1i/2,          0]
 
T^{-1} = 
[   0, - 1/2 - 1i/2, - 1/2 - 1i/2,            0,           1i,            0]
[   0,   1/2 - 1i/2,            0, - 1/2 + 1i/2, - 1/2 + 1i/2, - 1/2 - 1i/2]
[  1i,   1/2 - 1i/2,            0,            0, - 1/2 + 1i/2, - 1/2 - 1i/2]
[   0, - 1/2 + 1i/2, - 1/2 + 1i/2,            0,          -1i,            0]
[   0,   1/2 + 1i/2,            0, - 1/2 - 1i/2, - 1/2 - 1i/2, - 1/2 + 1i/2]
[ -1i,   1/2 + 1i/2,            0,            0, - 1/2 - 1i/2, - 1/2 + 1i/2]
 
AJ = 
[ 2 - 3i,      1,    

## Jordanizzazione (reale)
Nel solo caso che la matrice $A$ abbia autovettori complessi coniugati, possiamo portare la matrice $A_J$ comunque in forma reale. 

La costruzione non è molto diversa: se trovo un autovalore complesso $\lambda = \sigma + j\omega$ di $m_a(\lambda) = 1$, il suo miniblocco sarà, considerando anche il suo complesso coniugato:

$\begin{bmatrix} \sigma & \omega \\ -\omega & \sigma \end{bmatrix}$

mentre un blocco di molteplicità algebrica 2 sarà del tipo

$\begin{bmatrix} 
    \sigma & \omega & 1 & 0  \\ 
    -\omega & \sigma & 0 & 1 \\ 
    0 & 0 & \sigma & \omega  \\ 
    0 & 0 & -\omega & \sigma 
\end{bmatrix}$

mentre un blocco di molteplicità algebrica 3 sarà del tipo

$\begin{bmatrix} 
    \sigma  & \omega  & 1 & 0 & 0 & 0  \\ 
    -\omega & \sigma  & 0 & 1 & 0 & 0  \\ 
    0 & 0   & \sigma  & \omega & 1 & 0 \\ 
    0 & 0   & -\omega & \sigma & 0 & 1 \\
    0 & 0 & 0 & 0 &  \sigma & \omega   \\ 
    0 & 0 & 0 & 0 & -\omega & \sigma   \\
\end{bmatrix}$

a ricalcare la forma

$J_n(\lambda, \bar{\lambda}) = \begin{bmatrix} 
    \Lambda & \mathcal{I} & \mathcal{O} & \cdots & \mathcal{O}  \\ 
    \mathcal{O} & \Lambda & \mathcal{I} & \cdots & \mathcal{O}  \\ 
    \mathcal{O} & \mathcal{O} & \Lambda & \cdots & \mathcal{O}  \\ 
    \mathcal{O} & \mathcal{O} & \mathcal{O} & \cdots & \Lambda  \\ 
\end{bmatrix}; \quad
\Lambda = \begin{bmatrix} \sigma & \omega \\ -\omega & \sigma \end{bmatrix}; \quad
\mathcal{I} = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix}; \quad
\mathcal{O} = \begin{bmatrix} 0 & 0 \\ 0 & 0 \end{bmatrix}
$

Per quanto riguarda la matrice $T$, gli autovettori complessi coniugati legati agli autovalori $\lambda, \bar{\lambda}$ sono $v, \bar{v}$. Nella forma reale, associato al blocco avremo $\mathfrak{Re}(v), \mathfrak{Im}(v)$ che sono la parte reale ed immaginaria degli autovalori.

In [13]:
[jbr jbc] = size(jordanBlockSize);
[ar ac] = size(A);
n = ac;

AJR = [];
TR = [];

for i = 1:jbr
    
    % estraggo le informazioni che mi servono dalla tabella appena costruita
    lambda = jordanBlockSize(i,1);
    order = jordanBlockSize(i,2);
    
    if isreal(lambda)
        % se l'autovalore è reale allora procedo come al solito
        AJR = blkdiag(AJR, CreateJordanBlock(lambda, order));
        TR = [ TR, CreateJordanChain(A - lambda*eye(n), order, TR) ];
        
    elseif imag(lambda) > 0
        fprintf('Autovalore complesso %d + j%d... ordine %d\n', real(lambda), imag(lambda), order);
        % procedo nel creare il blocco intero complesso coniugato
        % questo branch genera blocchi di dimensione doppia perchè considera 
        % anche il complesso coniugato con parte immaginaria negativa
        AJR = blkdiag(AJR, CreateRealJordanBlock(lambda, order));
        TR = [ TR, CreateRealJordanChain(A - lambda*eye(n), order, TR) ];
        
    else
        % non faccio nulla, ho già processato nel caso precedente entrambi
        % gli autovalori complessi coniugati
    end
    
end

fprintf('TR = \n');
disp(TR);

fprintf('TR^{-1} = \n');
disp(inv(TR));

fprintf('AJR = \n');
disp(sym(AJR));

fprintf('A = \n');
disp(A);

fprintf('Controprova: TR * AJR * TR^{-1} = \n');
disp( TR * AJR * inv(TR) );

Autovalore complesso 2 + j3... ordine 3
TR = 
[    0,    0, -1/2, -1/2, 1/2, 1/2]
[ -1/2,  1/2, -1/2, -1/2,   1,   0]
[ -1/2, -1/2,  1/2,  1/2,  -1,   0]
[    0,    0,   -1,    0,   1,   0]
[ -1/2,  1/2,    0,    0,   0,   0]
[    0,    0, -1/2, -1/2,   0,   0]
 
TR^{-1} = 
[ 0, -1, -1,  0,  0,  0]
[ 0, -1, -1,  0,  2,  0]
[ 0,  1,  0, -1, -1, -1]
[ 0, -1,  0,  1,  1, -1]
[ 0,  1,  0,  0, -1, -1]
[ 2, -1,  0,  0,  1, -1]
 
AJR = 
[  2, 3,  1, 0,  0, 0]
[ -3, 2,  0, 1,  0, 0]
[  0, 0,  2, 3,  1, 0]
[  0, 0, -3, 2,  0, 1]
[  0, 0,  0, 0,  2, 3]
[  0, 0,  0, 0, -3, 2]
 
A = 
[  4,  0, 0, -3,  0,  1]
[  5,  4, 3, -2, -2, -2]
[ -5,  0, 2,  3, -3,  3]
[  6, -1, 0, -1,  1,  1]
[  0,  2, 3,  1,  0,  0]
[ -1,  3, 0, -3, -3,  3]
 
Controprova: TR * AJR * TR^{-1} = 
[  4,  0, 0, -3,  0,  1]
[  5,  4, 3, -2, -2, -2]
[ -5,  0, 2,  3, -3,  3]
[  6, -1, 0, -1,  1,  1]
[  0,  2, 3,  1,  0,  0]
[ -1,  3, 0, -3, -3,  3]
 


## Esponenziale delle matrici in forma (normale) di Jordan 

## Esponenziale delle matrici in forma reale di Jordan

Prendiamo la matrice in forma reale di Jordan $A_{JR}$. In questo caso la sua matrice esponenziale $e^{A_{JR}t}$ è più complicata. L'esponenziale di un blocco di Jordan complesso in forma reale è 

$J = \begin{bmatrix} \sigma & \omega \\ -\omega & \sigma \end{bmatrix}
\to
e^{Jt} = e^{\sigma t} \begin{bmatrix} \cos(\omega t) & \sin(\omega t) \\ -\sin(\omega t) & \cos(\omega t) \end{bmatrix}
$

mentre un blocco di molteplicità algebrica 2 sarà del tipo

$J = \begin{bmatrix} 
    \sigma & \omega & 1 & 0  \\ 
    -\omega & \sigma & 0 & 1 \\ 
    0 & 0 & \sigma & \omega  \\ 
    0 & 0 & -\omega & \sigma 
\end{bmatrix} 
\to
e^{Jt} 
= \exp \left(
\begin{bmatrix} 
    \Lambda & \mathcal{I} \\
    \mathcal{O} & \Lambda
\end{bmatrix} t \right)
= 
\begin{bmatrix} 
    e^{\Lambda t} & t e^{\Lambda t} \\
    \mathcal{O} & e^{\Lambda t}
\end{bmatrix}
= 
e^{\sigma t} 
\begin{bmatrix} 
     \cos(\omega t) & \sin(\omega t) &  t \cos(\omega t) & t \sin(\omega t) \\
    -\sin(\omega t) & \cos(\omega t) & -t \sin(\omega t) & t \cos(\omega t) \\
    0 & 0 &  \cos(\omega t) & \sin(\omega t) \\
    0 & 0 & -\sin(\omega t) & \cos(\omega t) \\
\end{bmatrix}
$

mentre un blocco di molteplicità algebrica 3 sarà del tipo

$J = \begin{bmatrix} 
    \sigma  & \omega  & 1 & 0 & 0 & 0  \\ 
    -\omega & \sigma  & 0 & 1 & 0 & 0  \\ 
    0 & 0   & \sigma  & \omega & 1 & 0 \\ 
    0 & 0   & -\omega & \sigma & 0 & 1 \\
    0 & 0 & 0 & 0 &  \sigma & \omega   \\ 
    0 & 0 & 0 & 0 & -\omega & \sigma   \\
\end{bmatrix}
\to 
e^{Jt} = \exp\left( 
    \begin{bmatrix}
        \Lambda     & \mathcal{I} & \mathcal{O} \\
        \mathcal{O} & \Lambda     & \mathcal{I} \\
        \mathcal{O} & \mathcal{O} & \Lambda     \\
    \end{bmatrix} t
\right)
= \begin{bmatrix}
        e^{\Lambda t} & t e^{\Lambda t} & \frac{t^2}{2} e^{\Lambda t} \\
        \mathcal{O}   & e^{\Lambda t}   & t e^{\Lambda t} \\
        \mathcal{O}   & \mathcal{O}     & e^{\Lambda t}    \\
    \end{bmatrix}
$